In [124]:
import pandas as pd
import numpy as np
from cataclop.ml.pipeline import factories
from cataclop.ml.exploration import random_race
from cataclop.ml import preprocessing

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 500)

NAN_FLAG = 10000

In [2]:
d = factories.Dataset.factory('default', {"from": '2013-05-10'})
#d = factories.Dataset.factory('default', {"from": '2018-05-10'})
d.load()
d.players.head()

,age,earnings,final_odds,final_odds_ref,handicap_distance,handicap_weight,herder_id,horse_id,imported_at_player,is_first_timer,is_racing,jockey_id,music,num,owner_id,placed_2_count,placed_3_count,placed_count,placed_dividend,placed_earnings,position,post_position,prev_year_earnings,race_count,race_id,time,trainer_id,victory_count,victory_earnings,winner_dividend,year_earnings,category,condition_age,condition_sex,declared_player_count,distance,imported_at_race,num_race,num_bis,prize,session_id,start_at,sub_category,date,hippodrome_id,imported_at_session,num_session,code,country,imported_at_hippo,name,hist_1_pos,hist_2_pos,hist_3_pos,hist_4_pos,hist_5_pos,hist_6_pos
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2,14787500,2.4,2.5,NaN,50.0,1.0,1,2018-05-29 01:16:26.626967,True,True,1,2p1p1p2p,1,1.0,2,0,2,0.0,6985000,4.0,3,0,4,1,NaN,1,2,14787500,0.0,14787500,PLAT,DEUX_ANS,FEMELLES,7,1600,2018-05-29 01:16:26.610753,1,NaN,300000,1,2017-10-01 14:12:00,GROUPE_I,2017-10-01,1,2018-05-29 01:16:26.605734,1,CHA,FRA,2018-05-29 01:16:26.605163,CHANTILLY,2.0,1.0,1.0,2.0,0.0,0.0
2,2,1623100,16.5,29.4,NaN,46.0,2.0,2,2018-05-29 01:16:26.737102,False,True,2,3p1p,2,2.0,0,1,1,350.0,1200000,1.0,6,0,2,1,NaN,2,1,1623100,1080.0,1623100,PLAT,DEUX_ANS,FEMELLES,7,1600,2018-05-29 01:16:26.610753,1,NaN,300000,1,2017-10-01 14:12:00,GROUPE_I,2017-10-01,1,2018-05-29 01:16:26.605734,1,CHA,FRA,2018-05-29 01:16:26.605163,CHANTILLY,3.0,1.0,0.0,0.0,0.0,0.0
3,2,8966500,19.8,27.9,NaN,48.0,3.0,3,2018-05-29 01:16:26.670131,False,True,3,4p1p1p1p,3,3.0,0,0,1,0.0,1998500,5.0,7,0,4,1,NaN,3,3,8966500,0.0,8966500,PLAT,DEUX_ANS,FEMELLES,7,1600,2018-05-29 01:16:26.610753,1,NaN,300000,1,2017-10-01 14:12:00,GROUPE_I,2017-10-01,1,2018-05-29 01:16:26.605734,1,CHA,FRA,2018-05-29 01:16:26.605163,CHANTILLY,4.0,1.0,1.0,1.0,0.0,0.0
4,2,5800000,9.0,14.1,NaN,44.0,4.0,4,2018-05-29 01:16:26.685392,False,True,4,1p4p1p1p,4,4.0,0,0,1,0.0,600000,3.0,4,0,4,1,NaN,4,3,5800000,0.0,5800000,PLAT,DEUX_ANS,FEMELLES,7,1600,2018-05-29 01:16:26.610753,1,NaN,300000,1,2017-10-01 14:12:00,GROUPE_I,2017-10-01,1,2018-05-29 01:16:26.605734,1,CHA,FRA,2018-05-29 01:16:26.605163,CHANTILLY,1.0,4.0,1.0,1.0,0.0,0.0
5,2,5350000,2.8,2.2,NaN,50.0,5.0,5,2018-05-29 01:16:26.740630,False,True,5,1p1p,5,5.0,0,0,0,180.0,0,2.0,2,0,2,1,NaN,5,2,5350000,0.0,5350000,PLAT,DEUX_ANS,FEMELLES,7,1600,2018-05-29 01:16:26.610753,1,NaN,300000,1,2017-10-01 14:12:00,GROUPE_I,2017-10-01,1,2018-05-29 01:16:26.605734,1,CHA,FRA,2018-05-29 01:16:26.605163,CHANTILLY,1.0,1.0,0.0,0.0,0.0,0.0


In [3]:
# keep the races we are interested in
df = d.players.groupby('race_id').filter(lambda race: True)
#df = d.players.groupby('race_id').filter(lambda race: race['race_count'].max() > 0)

races = df.groupby('race_id')

In [4]:
# append average speed, might be used as a target
df['speed'] = (df['distance'] / df['time']).fillna(1000)

# 
df['win'] = (df['position'] == 1).astype(np.float)

#
df['victory_earnings'] = np.log(1+df['victory_earnings'].fillna(0))
df['placed_earnings'] = np.log(1+df['placed_earnings'].fillna(0))
df['prev_year_earnings'] = np.log(1+df['prev_year_earnings'].fillna(0))
df['year_earnings'] = np.log(1+df['year_earnings'].fillna(0))

# append last odds inverse, equivalent to the estimated probability of winning 
df['final_odds_ref_inv'] = (1. / df['final_odds_ref']).fillna(0.)

In [5]:
# features we want to append stats relative to the race (mean, std ...)
agg_features = ['race_count', 
                'victory_count', 
                'placed_2_count', 
                'placed_3_count',
                'victory_earnings',
                'placed_earnings',
                'prev_year_earnings',
                'handicap_distance',
                'handicap_weight'
               ]
stats = races[agg_features].agg([np.mean, np.std, np.min, np.max])
# transform the 2 level columns index in 1 (victory_count, (mean, std)) -> (victory_count_mean, victory_count_std) 
stats.columns = ['_'.join(col) for col in stats.columns.values]

In [6]:
df = df.join(stats, how='left', on='race_id')

for f in agg_features:
    df['{}_r'.format(f)] = (df[f] - df['{}_mean'.format(f)]) / df['{}_std'.format(f)]
    
relative_features = ['{}_r'.format(f) for f in agg_features]

df[relative_features] = df[relative_features].replace([np.inf, -np.inf], np.nan)
df[relative_features] = df[relative_features].fillna(1000)

In [7]:
# append sorted odds of the race
odds = pd.DataFrame(columns=['odds_{:d}'.format(i) for i in range(20)], index=df.index)

races = df.groupby('race_id')
for (id, race) in races:
    odds_sorted = sorted(race['final_odds_ref'].values)[0:20]
    odds.loc[race.index, ['odds_{:d}'.format(i) for i, v in enumerate(odds_sorted)]] = odds_sorted

df = pd.concat([df,odds], axis=1)

df[['odds_{:d}'.format(i) for i in range(20)]] = df[['odds_{:d}'.format(i) for i in range(20)]].fillna(1000.0)

In [8]:
# keep only interesting races: at least one winner, with reference odds
df = df.groupby('race_id').filter(lambda r: r['position'].min() == 1 and r['odds_0'].min() != 1000 )
df = df.sort_index()

In [9]:
random_race(df, cols=['position', 'sub_category', 'num', 'final_odds', 'final_odds_ref', 'odds_0', 'odds_1', 'odds_2'] + agg_features, n=1)

position sub_category  num  final_odds  final_odds_ref  odds_0  \
race_id id                                                                     
827     5757       5.0   A_RECLAMER    1        19.4            23.6     2.7   
        5758       1.0   A_RECLAMER    2         2.5             2.7     2.7   
        5759       4.0   A_RECLAMER    3         4.3             5.8     2.7   
        5760       3.0   A_RECLAMER    4         4.0             8.5     2.7   
        5761       NaN   A_RECLAMER    5         5.6             4.7     2.7   
        5762       2.0   A_RECLAMER    6        15.0            23.0     2.7   
        5763       6.0   A_RECLAMER    7         5.5             5.7     2.7   
        5764       7.0   A_RECLAMER    8        27.9            22.3     2.7   

              odds_1  odds_2  race_count  victory_count  placed_2_count  \
race_id id                                                                
827     5757     4.7     5.7          14              2               0   
        5758     4.7     5.7           7              1               1   
        5759     4.7     5.7          10              2               5   
        5760     4.7     5.7           4              1               1   
        5761     4.7     5.7           1              0               0   
        5762     4.7     5.7           3              1               0   
        5763     4.7     5.7          15              1               4   
        5764     4.7     5.7           7              0               1   

              placed_3_count  victory_earnings  placed_earnings  \
race_id id                                                        
827     5757               2         15.182877        14.301019   
        5758               3         14.859315        14.340240   
        5759               1         15.539170        14.964499   
        5760               1         14.190517        13.493928   
        5761               0         12.506181        12.506181   
        5762               0         13.764218         0.000000   
        5763               1         15.154451        14.868079   
        5764               1         13.329379        13.329379   

              prev_year_earnings  handicap_distance  handicap_weight  
race_id id                                                            
827     5757                 0.0                NaN             35.0  
        5758                 0.0                NaN             36.0  
        5759                 0.0                NaN             36.0  
        5760                 0.0                NaN             33.0  
        5761                 0.0                NaN              NaN  
        5762                 0.0                NaN             31.0  
        5763                 0.0                NaN             31.0  
        5764                 0.0                NaN             29.0

In [10]:
features = ['prize', 'declared_player_count']

#features += ['final_odds_ref']

features += ['odds_{:d}'.format(i) for i in range(10)]

features += ['hist_{}_pos'.format(i+1) for i in range(6)]

features += agg_features

# too lazy to write a comprehension
for f in agg_features:
    for s in ['amin', 'amax', 'std', 'mean', 'r']:
        features.append('{}_{}'.format(f, s))
        
# unique features list
features = list(set(features))

# this will be transformed in the model training
categorical_features = ['category', 'sub_category', 'country']

In [11]:
sorted(features)

['declared_player_count',
 'handicap_distance',
 'handicap_distance_amax',
 'handicap_distance_amin',
 'handicap_distance_mean',
 'handicap_distance_r',
 'handicap_distance_std',
 'handicap_weight',
 'handicap_weight_amax',
 'handicap_weight_amin',
 'handicap_weight_mean',
 'handicap_weight_r',
 'handicap_weight_std',
 'hist_1_pos',
 'hist_2_pos',
 'hist_3_pos',
 'hist_4_pos',
 'hist_5_pos',
 'hist_6_pos',
 'odds_0',
 'odds_1',
 'odds_2',
 'odds_3',
 'odds_4',
 'odds_5',
 'odds_6',
 'odds_7',
 'odds_8',
 'odds_9',
 'placed_2_count',
 'placed_2_count_amax',
 'placed_2_count_amin',
 'placed_2_count_mean',
 'placed_2_count_r',
 'placed_2_count_std',
 'placed_3_count',
 'placed_3_count_amax',
 'placed_3_count_amin',
 'placed_3_count_mean',
 'placed_3_count_r',
 'placed_3_count_std',
 'placed_earnings',
 'placed_earnings_amax',
 'placed_earnings_amin',
 'placed_earnings_mean',
 'placed_earnings_r',
 'placed_earnings_std',
 'prev_year_earnings',
 'prev_year_earnings_amax',
 'prev_year_earnin

In [12]:
N_TARGETS = 1

df['target'] = df['winner_dividend'] / 100.
#df.loc[df['target'] > N_TARGETS, 'target'] = np.log(1.+N_TARGETS)
df['target'].fillna(0, inplace=True)

# to test the training is working we can append the target 
#features.append('target')

#for f in features:
#    print('{} \t: {}'.format(f, df[f].corr(df['target'])))

In [84]:
from sklearn.model_selection import KFold, GroupKFold
from sklearn.metrics import log_loss, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler, Normalizer, StandardScaler

# 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier, MLPRegressor

groups = df['race_id'].values

group_kfold = GroupKFold(n_splits=2)

splits = list(group_kfold.split(df.values, df['position'].values, groups))

models = [
    {
        'name': 'knn_10',
        'model': KNeighborsRegressor(n_neighbors=10),
        'scaler': True
    },
    {
        'name': 'knn_30',
        'model': KNeighborsRegressor(n_neighbors=30),
        'scaler': True
    },
    {
        'name': 'mlp',
        'model': MLPRegressor(),
        'scaler': True
    },
    {
        'name': 'rf_10',
        'model': RandomForestRegressor(n_estimators=10),
        'scaler': False
    },
    {
        'name': 'rf_100',
        'model': RandomForestRegressor(n_estimators=100),
        'scaler': False
    },
]

for model in models:
    for i in range(N_TARGETS):
        df['pred_{}_{}'.format(model['name'], i+1)] = 0.0


In [85]:
df.loc[:, features] = df.loc[:, features].fillna(NAN_FLAG)
df = df.reset_index()

In [86]:
for train_index, test_index in splits:
    
    for model in models:

        X_train = df[features].iloc[train_index].copy()
        y_train = df['target'].iloc[train_index]

        dummies = preprocessing.get_dummies(df.iloc[train_index], categorical_features)
        X_train = pd.concat([X_train, preprocessing.get_dummy_values(df.iloc[train_index], dummies)], axis=1)

        idx = (df.iloc[train_index]['target'] != NAN_FLAG) & (df.iloc[train_index]['category'] == 'PLAT') & (df.iloc[train_index]['final_odds_ref'] < 30) & ((df.iloc[train_index]['position'] == 1) | (df.iloc[train_index]['position'] == 4)) 
        X_train = X_train[ idx ]
        y_train = df['target'].iloc[train_index][ idx ]

        X_test = df[features].iloc[test_index].copy()
        y_test = df['target'].iloc[test_index]
    
        X_test = pd.concat([X_test, preprocessing.get_dummy_values(df.iloc[test_index], dummies)], axis=1)
    
        X_train = X_train.values
        X_test = X_test.values
    
        idx = df.iloc[test_index].index

        if model['scaler']:
            scaler = StandardScaler()
            scaler.fit(X_train)

            X_train = scaler.transform(X_train)
            X_test = scaler.transform(X_test)

        clf = model['model']
        clf.fit(X_train, y_train.values)
        
        if N_TARGETS > 1:
            
            p = clf.predict_proba(X_test)

            print( model['name'], log_loss(y_test.values, p) )

            for i in range(N_TARGETS):
                df.loc[idx, 'pred_{}_{}'.format(model['name'], i+1)] = p[:, list(clf.classes_).index(i+1)]
            
        else:

            p = clf.predict(X_test)
        
            df.loc[idx, 'pred_{}_1'.format(model['name'])] = p
        
            print( model['name'], mean_absolute_error(y_test.values, p) )


knn_10 4.280148634193848
knn_30 4.1855774276120075


/Users/pourquoi/.local/share/virtualenvs/cataclop-django-M9amAh3O/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


mlp 5.747406569810625
rf_10 6.426814250211156
rf_100 6.204073793375081
knn_10 4.439607260884141
knn_30 4.308194890134159


/Users/pourquoi/.local/share/virtualenvs/cataclop-django-M9amAh3O/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


mlp 5.702100394708783
rf_10 5.781893868830001
rf_100 5.931691809320204


In [105]:
random_race(df, ['sub_category', 'num', 'position', 'final_odds', 'pred_rf_10_1', 'pred_rf_100_1', 'pred_knn_10_1', 'pred_knn_30_1', 'pred_mlp_1'])

sub_category  num  position  final_odds  pred_rf_10_1  \
race_id                                                               
13480   82735    AUTOSTART    1       5.0         4.5          7.76   
        82736    AUTOSTART    2       4.0         5.7         13.40   
        82737    AUTOSTART    3       7.0        34.6          8.58   
        82738    AUTOSTART    4       6.0        35.9          9.72   
        82739    AUTOSTART    5       2.0         7.1          4.87   
        82740    AUTOSTART    6       9.0        21.4         13.40   
        82741    AUTOSTART    7       3.0        32.6          6.91   
        82742    AUTOSTART    8       NaN        33.6          7.75   
        82743    AUTOSTART    9       1.0         3.7          5.51   
        82744    AUTOSTART   10       8.0         5.3          9.00   
        82745    AUTOSTART   11      10.0        62.4          8.45   

               pred_rf_100_1  pred_knn_10_1  pred_knn_30_1  pred_mlp_1  
race_id                                                                 
13480   82735       5.427583           5.39       3.876667   14.646859  
        82736      16.072250           5.20       3.763333   11.908235  
        82737       6.067750           5.18       3.933333   11.186457  
        82738       6.276583           5.81       4.380000   17.699618  
        82739       4.869583           5.81       4.380000   13.213827  
        82740      16.072250           5.20       3.763333   11.908235  
        82741       5.919583           5.81       4.380000   17.570291  
        82742       6.739000           5.39       3.880000   12.845064  
        82743       6.672167           6.60       3.880000   13.249691  
        82744       7.338000           5.39       3.880000   10.430254  
        82745       6.872000           5.39       3.880000   11.598038

In [106]:
for model in models:
    clf = model['model']
    if hasattr(clf, 'feature_importances_'):
        fi = pd.Series(clf.feature_importances_, index=features+preprocessing.get_dummy_features(dummies))
        print(fi.sort_values(ascending=False))
        break

victory_earnings_r                              0.052870
handicap_weight_r                               0.035774
race_count_r                                    0.033669
odds_9                                          0.030055
odds_8                                          0.028579
placed_2_count_r                                0.028267
placed_earnings_r                               0.028191
placed_3_count_r                                0.028185
hist_1_pos                                      0.027114
prev_year_earnings_r                            0.025560
victory_earnings_std                            0.023134
victory_count_r                                 0.022807
odds_0                                          0.022150
odds_7                                          0.018539
hist_5_pos                                      0.018423
odds_3                                          0.018291
placed_earnings_std                             0.018107
odds_1                         

In [108]:
targets = ['pred_knn_10_1', 'pred_knn_30_1', 'pred_rf_10_1', 'pred_rf_100_1', 'pred_mlp_1']

df[targets].describe()

,pred_knn_10_1,pred_knn_30_1,pred_rf_10_1,pred_rf_100_1,pred_mlp_1
count,125274.000000,125274.000000,125274.000000,125274.000000,125274.000000
mean,4.195131,4.092608,6.031720,6.030782,4.920950
std,2.202443,1.401148,4.520471,3.329261,4.610731
min,0.000000,0.000000,0.000000,0.000000,-38.343307
25%,2.660000,3.146667,3.050000,3.780000,2.313855
50%,3.840000,3.963333,4.890000,5.281000,4.819669
75%,5.340000,4.920000,7.720000,7.408821,7.551054
max,20.650000,12.136667,52.910000,37.189000,43.047547


In [109]:
races = df.sort_values('start_at').groupby('race_id')

bets = []

stash = 500

for (id, race) in races:
    
    candidate_bets = []
    
    nums = []
    
    for target in targets:

        r = race.sort_values(by=target, ascending=False)

        odds = r['final_odds_ref'].min()
        nth = len(r[r['final_odds_ref']<odds])+1

        N = 1

        for n in range(N):

            if len(r) <= N:
                continue

            player = r.iloc[n]

            if player['num'] in nums:
                continue

            odds = player['final_odds_ref']

            if odds > 20:
                break

            nth = (r['final_odds_ref']<odds).sum()+1

            bet = 1.

            profit = player['winner_dividend']/100.0 * bet - bet

            stash = stash + profit
            
            row = [id, player['date'], player['num'], odds, player['final_odds'], target, player[target], bet, profit]
            
            for f in features:
                row.append(player[f])
            for f in categorical_features:
                row.append(player[f])

            candidate_bets.append( row )

            nums.append(player['num'])
            
            break
    
    if len(candidate_bets) == 1:
        bets += candidate_bets
        
        
cols = ['id', 'date', 'num', 'odds_ref', 'odds_final', 'target', 'pred', 'bet', 'profit'] + features + categorical_features

bets = pd.DataFrame(bets, columns=cols)

bets.index = bets['date']

bets = bets.sort_index()

bets['bets'] = bets['bet'].cumsum()
bets['stash'] = bets['profit'].cumsum()

In [134]:
bets[(bets['target'] == 'pred_rf_100_1') & (bets['odds_ref'] > 5) & (bets['category'] == 'PLAT')]

,id,date,num,odds_ref,odds_final,target,pred,bet,profit,handicap_distance,placed_2_count_amax,prev_year_earnings_r,odds_6,odds_1,odds_3,victory_count_r,victory_count_std,declared_player_count,victory_count,hist_6_pos,placed_earnings,victory_earnings_std,placed_earnings_amax,placed_earnings_amin,handicap_weight_amax,handicap_weight_mean,odds_5,odds_4,handicap_distance_amax,prev_year_earnings_mean,race_count,victory_earnings_mean,race_count_amax,handicap_weight_std,odds_2,prev_year_earnings_amax,handicap_distance_std,placed_2_count_mean,victory_earnings_amax,handicap_distance_amin,placed_2_count_std,odds_0,prev_year_earnings,placed_3_count_std,placed_3_count_amax,handicap_distance_r,hist_1_pos,hist_2_pos,placed_earnings_mean,hist_5_pos,placed_3_count_amin,handicap_weight_amin,prize,hist_3_pos,placed_2_count_r,placed_3_count_r,hist_4_pos,race_count_amin,odds_8,odds_7,victory_earnings,race_count_mean,victory_count_amax,victory_count_mean,placed_2_count_amin,placed_earnings_std,odds_9,placed_3_count,race_count_std,race_count_r,placed_2_count,victory_count_amin,handicap_weight,handicap_distance_mean,victory_earnings_r,victory_earnings_amin,placed_earnings_r,prev_year_earnings_amin,handicap_weight_r,placed_3_count_mean,prev_year_earnings_std,category,sub_category,country,bets,stash
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-01-13,12821,2017-01-13,8,17.0,17.9,pred_rf_100_1,5.836000,1.0,-1.0,10000.0,2,0.558615,29.2,4.0,13.9,1000.000000,0.000000,10,0,0.0,11.573089,4.800460,12.597104,0.000000,29.0,25.125000,17.0,16.9,10000.0,8.899456,5,8.915870,15,3.642507,6.3,12.528942,10000.0,0.500000,12.597104,10000.0,0.849837,2.1,11.573437,0.843274,2,1000.0,8.0,2.0,8.915870,0.0,0,19.0,4523,9.0,0.588348,-0.474342,8.0,1,42.0,38.4,11.573089,7.400000,0,0.000000,0,4.800460,48.8,0,4.647580,-0.516398,1,0,28.0,10000.0,0.553534,0.000000,0.553534,0.000000,0.789292,0.400000,4.786806,PLAT,COURSE_A_CONDITIONS,ZAF,81.0,-34.0
2017-01-18,13006,2017-01-18,13,18.3,23.5,pred_rf_100_1,8.411000,1.0,-1.0,10000.0,2,0.685935,11.6,7.0,8.7,-0.663212,0.646206,14,0,3.0,14.433396,6.568687,14.433396,0.000000,33.0,29.785714,10.2,9.5,10000.0,9.935435,14,9.951909,14,1.888368,7.9,14.580979,10000.0,0.428571,14.703402,10000.0,0.646206,4.8,14.433396,1.292412,4,1000.0,9.0,9.0,9.592367,7.0,0,27.0,16000,6.0,0.884282,1.658029,5.0,3,20.1,18.3,14.433396,7.928571,2,0.428571,0,6.329117,23.3,3,3.852358,1.576029,1,0,27.0,10000.0,0.682250,0.000000,0.764882,0.000000,-1.475197,0.857143,6.557418,PLAT,HANDICAP,FRA,116.0,-24.3
2017-01-20,13087,2017-01-20,8,17.9,36.7,pred_rf_100_1,11.833000,1.0,-1.0,10000.0,7,-3.827557,16.1,5.8,10.1,-1.499167,1.765686,17,1,0.0,0.000000,0.819528,15.922081,0.000000,36.0,34.117647,14.5,11.6,10000.0,13.721866,3,15.728813,48,1.615640,7.0,15.481518,10000.0,3.705882,16.537201,10000.0,2.284861,5.6,0.000000,2.222876,8,1000.0,8.0,6.0,14.313268,0.0,0,31.0,26000,1.0,-1.621929,-1.455454,0.0,3,17.9,17.0,13.017005,27.588235,7,3.647059,0,3.716270,21.6,0,11.869387,-2.071567,0,1,35.0,10000.0,-3.308987,13.017005,-3.851515,0.000000,0.546132,3.235294,3.585019,PLAT,HANDICAP_DIVISE,FRA,128.0,-32.5
2017-01-26,13330,2017-01-26,11,18.5,27.9,pred_rf_100_1,14.994000,1.0,-1.0,10000.0,4,-2.311430,18.5,7.3,15.8,-0.670982,0.851631,14,0,0.0,0.000000,5.217532,14.753563,0.000000,30.0,25.714286,18.4,16.8,10000.0,11.861739,3,12.028671,26,3.099096,11.3,15.243427,10000.0,1.214286,15.319144,10000.0,1.251373,2.0,0.000000,2.006856,7,1000.0,6.0,9.0,11.830098,0.0,0,18.0,19000,8.0,-0.970363,-0.889807,0.0,3,24.4,20.3,0.000000,14.642857,3,0.571429,0,5.106413,28.8,0,8.063280,-1.443936,0,0,25.0,10000.0,-2.305433,0.000000,-2.316714,0.000000,-0.230482,1.785714,5.131776,PLAT,HANDICAP,FRA,156.0,-32.6
2017-01-26,13340,2017-01-26,2,7.0,5.6,pred_rf_100_1,13.287000,1.0,-1.0,10000.0,16,0.586371,14.5,4.9,9.1,-0.760322,3.594970,15,2,0.0,17.390448,1.043881,19.387702,14.590699,49.0,47.066667,14.3,13.3,10000.0,13.524921,17,16.942276,70,1.579632,7.0,16.770422,10000.0,4.733333,19.3

In [110]:
bets

,id,date,num,odds_ref,odds_final,target,pred,bet,profit,handicap_distance,placed_2_count_amax,prev_year_earnings_r,odds_6,odds_1,odds_3,victory_count_r,victory_count_std,declared_player_count,victory_count,hist_6_pos,placed_earnings,victory_earnings_std,placed_earnings_amax,placed_earnings_amin,handicap_weight_amax,handicap_weight_mean,odds_5,odds_4,handicap_distance_amax,prev_year_earnings_mean,race_count,victory_earnings_mean,race_count_amax,handicap_weight_std,odds_2,prev_year_earnings_amax,handicap_distance_std,placed_2_count_mean,victory_earnings_amax,handicap_distance_amin,placed_2_count_std,odds_0,prev_year_earnings,placed_3_count_std,placed_3_count_amax,handicap_distance_r,hist_1_pos,hist_2_pos,placed_earnings_mean,hist_5_pos,placed_3_count_amin,handicap_weight_amin,prize,hist_3_pos,placed_2_count_r,placed_3_count_r,hist_4_pos,race_count_amin,odds_8,odds_7,victory_earnings,race_count_mean,victory_count_amax,victory_count_mean,placed_2_count_amin,placed_earnings_std,odds_9,placed_3_count,race_count_std,race_count_r,placed_2_count,victory_count_amin,handicap_weight,handicap_distance_mean,victory_earnings_r,victory_earnings_amin,placed_earnings_r,prev_year_earnings_amin,handicap_weight_r,placed_3_count_mean,prev_year_earnings_std,category,sub_category,country,bets,stash
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-01-01,12360,2017-01-01,1,7.6,8.7,pred_knn_30_1,4.366667,1.0,-1.0,2100.0,7,0.487782,18.0,5.7,7.6,-1.015092,1.818706,13,2,2.0,16.076961,0.141882,16.079355,14.222974,10000.0,10000.000000,10.5,9.7,2100.0,14.995485,21,16.171495,41,10000.000000,7.5,15.761278,0.0,3.153846,16.397015,2100.0,1.818706,3.3,15.224300,2.586949,8,1000.0,0.0,3.0,15.342984,3.0,0,10000.0,70000,9.0,2.114775,1.070462,3.0,16,22.8,21.4,16.313827,25.538462,7,3.846154,0,0.607951,23.2,6,9.116061,-0.497853,7,1,10000.0,2100.0,1.003165,15.918058,1.207295,13.935071,1000.000000,3.230769,0.469094,ATTELE,AUTOSTART,FRA,1.0,-1.0
2017-01-01,12362,2017-01-01,8,9.6,6.8,pred_knn_10_1,4.830000,1.0,-1.0,2100.0,7,-1.084406,17.3,9.6,10.7,-0.843919,3.489012,18,1,2.0,13.830400,0.396578,16.393148,0.000000,10000.0,10000.000000,12.7,11.9,2100.0,8.832587,4,16.685678,65,10000.000000,10.2,16.822183,0.0,1.722222,17.025911,2100.0,2.052418,2.3,0.000000,1.789585,6,1000.0,7.0,7.0,13.472210,5.0,0,10000.0,90000,1.0,-0.839119,-0.869227,4.0,0,21.4,18.1,16.838084,20.166667,12,3.944444,0,4.983784,22.3,0,17.119820,-0.944325,0,0,10000.0,2100.0,0.384302,15.541273,0.071871,0.000000,1000.000000,1.555556,8.145094,ATTELE,EUROPEENNE_AUTOSTART,FRA,2.0,-2.0
2017-01-01,12363,2017-01-01,7,18.3,25.0,pred_rf_100_1,17.804000,1.0,-1.0,2100.0,18,0.319630,25.5,7.6,18.3,-0.652863,9.190296,14,7,9.0,16.898108,0.732493,18.970577,13.774690,10000.0,10000.000000,23.3,22.6,2100.0,15.696755,33,18.376307,90,10000.000000,15.7,17.981424,0.0,7.857143,19.872552,2100.0,4.990100,1.4,17.162428,4.496641,13,1000.0,7.0,3.0,17.235540,4.0,0,10000.0,120000,1.0,-0.572562,-1.175481,6.0,14,38.8,34.6,18.340909,54.857143,31,13.000000,0,1.242417,58.8,1,23.579186,-0.926968,5,1,10000.0,2100.0,-0.048326,17.194428,-0.271593,0.000000,1000.000000,6.285714,4.585524,ATTELE,INTERNATIONALE_AUTOSTART,FRA,3.0,-3.0
2017-01-01,12368,2017-01-01,7,6.8,6.6,pred_knn_10_1,6.620000,1.0,-1.0,10000.0,1,-1.196601,28.4,5.0,5.6,1000.000000,0.000000,8,0,0.0,0.000000,6.450744,13.458837,0.000000,32.0,31.000000,13.0,6.8,10000.0,7.718970,0,7.718970,9,1.000000,5.4,13.458837,10000.0,0.250000,13.458837,10000.0,0.462910,2.8,0.000000,0.353553,1,1000.0,0.0,0.0,7.718970,0.0,0,30.0,16000,0.0,-0.540062,-0.353553,0.0,0,1000.0,1000.0,0.000000,3.000000,0,0.000000,0,6.450744,1000.0,0,2.927700,-1.024695,0,0,10000.0,10000.0,-1.196601,0.000000,-1.196601,0.000000,1000.000000,0.125000,6.450744,PLAT,COURSE_A_CONDITIONS,FRA,4.0,-4.0
2017-01-01,12377,2017-01-01,4,18.7,22.0,pred_knn_30_1,5.123333,1.0,-1.0,2100.0,0,1000.000000,42.7,7.8,20.4,1000.000000,0.000000,13,0,8.0,0.000000,0.121868,0.000000,0.000000,10000.0,10000.000000,39.4,35.0,

In [136]:
bets[(bets['target'] == 'pred_rf_100_1') & (bets['odds_ref'] > 5) & (bets['category'] == 'PLAT')]['profit'].describe()

count    96.000000
mean      0.593750
std       6.113984
min      -1.000000
25%      -1.000000
50%      -1.000000
75%      -1.000000
max      35.500000
Name: profit, dtype: float64

In [22]:
len(bets)

20742